In [1]:
# Add names to pairs!

In [2]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from TransformerGlow import AdamWeightDecayOptimizer, FactorTrainer
import numpy as np
import torch
import random
import pandas as pd
import pickle
# import os, sys
# import time
# import datetime
# import torch.nn.functional as F
# from torch import nn



# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [3]:
def get_examples1(embeddings, gen_labels, attribute_ids, rand_seed=2, train=True):
    random.seed(rand_seed)
    female_embeddings = []
    male_embeddings = []
    female_gen_labels = []
    male_gen_labels = []
    female_attribute_ids = []
    male_attribute_ids = []
    female_logits = []
    male_logits = []

    # print(len(gen_labels))
    for key,i in enumerate(gen_labels):
        # print(i)
        if i==0:
            female_embeddings.append(embeddings[key])
            female_gen_labels.append(i)
            female_attribute_ids.append(attribute_ids[key])
            # female_logits.append(logits[key])
            # print(key)
        else:
            male_embeddings.append(embeddings[key])
            # print(embeddings[key])
            male_gen_labels.append(i)
            male_attribute_ids.append(attribute_ids[key])
            # male_logits.append(logits[key])

    female_embeddings = torch.tensor(np.array(female_embeddings))
    female_attribute_ids = torch.tensor(np.array(female_attribute_ids))
    female_gen_labels = torch.tensor(np.array(female_gen_labels))
    # female_logits = torch.tensor(np.array(female_logits))
    
    male_embeddings = torch.tensor(np.array(male_embeddings))
    male_attribute_ids = torch.tensor(np.array(male_attribute_ids))
    male_gen_labels = torch.tensor(np.array(male_gen_labels))
    # male_logits = torch.tensor(np.array(male_logits))
    
    return female_embeddings, female_attribute_ids, female_gen_labels, male_embeddings, male_attribute_ids, male_gen_labels

In [4]:
def get_examples(embeddings, gen_labels, attribute_ids, logits, rand_seed, train=True):
    random.seed(rand_seed)
    female_embeddings = []
    male_embeddings = []
    female_gen_labels = []
    male_gen_labels = []
    female_attribute_ids = []
    male_attribute_ids = []
    female_logits = []
    male_logits = []

    # print(len(gen_labels))
    for key,i in enumerate(gen_labels):
        # print(i)
        if i==0:
            if logits[key]<-0:
                female_embeddings.append(embeddings[key])
                female_gen_labels.append(i)
                female_attribute_ids.append(attribute_ids[key])
                female_logits.append(logits[key])
                # print(key)
        else:
            if logits[key]>0:
                male_embeddings.append(embeddings[key])
                # print(embeddings[key])
                male_gen_labels.append(i)
                male_attribute_ids.append(attribute_ids[key])
                male_logits.append(logits[key])

    female_embeddings = torch.tensor(np.array(female_embeddings))
    female_attribute_ids = torch.tensor(np.array(female_attribute_ids))
    female_gen_labels = torch.tensor(np.array(female_gen_labels))
    female_logits = torch.tensor(np.array(female_logits))
    
    male_embeddings = torch.tensor(np.array(male_embeddings))
    male_attribute_ids = torch.tensor(np.array(male_attribute_ids))
    male_gen_labels = torch.tensor(np.array(male_gen_labels))
    male_logits = torch.tensor(np.array(male_logits))
    
    return female_embeddings, female_attribute_ids, female_gen_labels, female_logits, male_embeddings, male_attribute_ids, male_gen_labels, male_logits

In [5]:
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/embeddings.pkl', 'rb') as f:
#         embeddings = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/gender.pkl', 'rb') as f:
#         gen_labels = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/token_ids.pkl', 'rb') as f:
#         attribute_ids = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/logits.pkl', 'rb') as f:
#         logits = pickle.load(f)

In [6]:
# import json

# with open('eval_gender_pairs.json') as f:
#     d = json.load(f)
# male_eval_words = [i[0] for i in d]
# female_eval_words = [i[1] for i in d]

In [7]:
# import random
# train_limit = int(0.99*len(embeddings))
# random.Random(4).shuffle(embeddings)
# random.Random(4).shuffle(gen_labels)
# random.Random(4).shuffle(attribute_ids)
# embeddings = embeddings[:train_limit]
# gen_labels = gen_labels[:train_limit]
# attribute_ids = attribute_ids[:train_limit]
# logits = logits[:train_limit]

# # test_embeddings = embeddings[train_limit:]
# # test_gen_labels = gen_labels[train_limit:]
# # test_attribute_ids = attribute_ids[train_limit:]
# # test_logits = logits[train_limit:]


In [8]:
attribute_pair = "african_european"
model_path = "bertflow_model_"+attribute_pair
bertflow = torch.load( model_path+'/bertflow_rho_0999_factors_6.pth')

#########TEST#################

In [9]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocabs = tokenizer.get_vocab()
vv = dict((v,k) for k,v in vocabs.items())

embedding_decoder = BertForMaskedLM.from_pretrained("bert-base-uncased").cls.cuda()
def decode_emb(emb):
    embedding_decoder.eval()
    with torch.no_grad():
        logits_list = embedding_decoder(torch.tensor(emb))
        id_list = torch.argmax(logits_list, dim=1)
        return id_list
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# from torch.utils.data import TensorDataset

# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/embeddings_'+str(2779)+'.pkl', 'rb') as f:
#     embeddings1 = pickle.load(f)
# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/gender_'+str(2779)+'.pkl', 'rb') as f:
#         gen_labels1 = pickle.load(f)
# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/token_ids_'+str(2779)+'.pkl', 'rb') as f:
#         attribute_ids1 = pickle.load(f)
# # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/logits_'+str(2779)+'.pkl', 'rb') as f:
# #         logits1 = pickle.load(f)


# female_embeddings1, female_attribute_ids1, female_gen_labels1, male_embeddings1, male_attribute_ids1, male_gen_labels1 = get_examples1(embeddings1, gen_labels1, attribute_ids1, train=False)
# # female_data_pairs_test1 = TensorDataset(female_embeddings1, female_attribute_ids1, female_gen_labels1)
# male_data_pairs_test1 = TensorDataset(male_embeddings1, male_attribute_ids1, male_gen_labels1)

# # female_dataloader1 = DataLoader(
# #             female_data_pairs_test1,  # The training samples.
# #             sampler = None, #RandomSampler(female_data_pairs_test), # Select batches randomly
# #             batch_size = 20 # Trains with this batch size.
# #         )

# male_dataloader1 = DataLoader(
#             male_data_pairs_test1,  # The training samples.
#             sampler = None, #RandomSampler(male_data_pairs_test), # Select batches randomly
#             batch_size = 20 # Trains with this batch size.
#         )





# bertflow.eval()

# female_z_list = []
# male_z_list = []
# f_embs = []
# m_embs = []
# f_ids = []
# m_ids = []
# f_logits = []
# m_logits = []

# # For each batch of training data...
# bertflow.eval()
# with torch.no_grad():
#     # for step, f_batch in enumerate(female_dataloader1):
#     #     # print(f_batch.shape)
#     #     embs = f_batch[0]
#     #     ids = f_batch[1]
#     #     # logit_v = f_batch[3]
#     #     # print(embs.shape)
#     #     female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#     #     female_z_list.append(female_z)
#     #     f_embs.append(embs)
#     #     f_ids.append(ids)
#     #     # f_logits.append(logit_v)
#     for step, m_batch in enumerate(male_dataloader1):
#         embs = m_batch[0]
#         ids = m_batch[1]
#         # logit_v = m_batch[3]
#         # gen = m_batch[2]
#         male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#         male_z_list.append(male_z)
#         m_embs.append(embs)
#         m_ids.append(ids)
#         # m_logits.append(logit_v)        


# sum_fem = 0
# count = 0
# for j in range(len(male_z_list)):
#     # print("############", j)
#     for k in range(len(male_z_list[j][0])):
#         sum_fem = sum_fem + male_z_list[j][0][k]
#         count+=1
# avg_sum_fem = sum_fem/count


In [11]:
# import copy
# #########generate embeddings in gaussian space#################
# def gen_flow_embs(female_dataloader, male_dataloader):
#     bertflow.eval()
    
#     female_z_list = []
#     male_z_list = []
#     f_embs = []
#     m_embs = []
#     f_ids = []
#     m_ids = []
#     f_logits = []
#     m_logits = []
    
#     # For each batch of training data...
#     bertflow.eval()
#     with torch.no_grad():
#         for step, f_batch in enumerate(female_dataloader):
#             # print(f_batch.shape)
#             embs = f_batch[0]
#             ids = f_batch[1]
#             logit_v = f_batch[3]
#             # print(embs.shape)
#             female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#             female_z_list.append(female_z)
#             f_embs.append(embs)
#             f_ids.append(ids)
#             f_logits.append(logit_v)
#         # for step, m_batch in enumerate(male_dataloader):
#         #     embs = m_batch[0]
#         #     ids = m_batch[1]
#         #     logit_v = m_batch[3]
#         #     # gen = m_batch[2]
#         #     male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#         #     male_z_list.append(male_z)
#         #     m_embs.append(embs)
#         #     m_ids.append(ids)
#         #     m_logits.append(logit_v)

#     new_female_z_list = copy.deepcopy(female_z_list)
#     # new_male_z_list = copy.deepcopy(male_z_list)

#     gender_transfer_count = 0
#     semantic_transfer_count = 0
#     non_conversion_count = 0
#     original_old_word = []
#     old_word = []
#     old_id = []
#     new_word = []
#     new_id = []
#     gender = []
#     logit_vals = []

#     # sum_fem = 0
#     # count = 0
#     # for j in range(len(male_z_list)):
#     #     # print("############", j)
#     #     for k in range(len(male_z_list[j][0])):
#     #         sum_fem = sum_fem + male_z_list[j][0][k]
#     #         count+=1
#     # avg_sum_fem = sum_fem/count

#     for j in range(len(female_z_list)):
#         for k in range(len(female_z_list[j][0])):
#         # part1 = torch.tensor([female_z[0][0][15].cpu().detach().numpy() for i in female_z[0][0]])
#             # new_male_z_list[j][0][0][k] = female_z_list[j][0][0][k]
#             new_female_z_list[j][0][k] = avg_sum_fem
        
            
#         eee = bertflow(new_female_z_list[j], reverse=True)
#         eee = torch.squeeze(eee)
#         ffff = decode_emb(eee)
#         ffff
#         for key,i in enumerate(decode_emb(f_embs[j][:,:].to(device))):
#             # if f_logits[j][key].item() < -10:
#             original_old_word.append(vv[f_ids[j][key].item()])
#             old_word.append(vv[i.item()])
#             old_id.append(i.item())
#             new_word.append(vv[ffff[key].item()])
#             new_id.append(ffff[key].item())
#             gender.append(1)
#             logit_vals.append(f_logits[j][key].item())
    
#     return original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals

In [12]:
def gen_flow_embs1(male_dataloader):
    bertflow.eval()
    
    female_z_list = []
    male_z_list = []
    f_embs = []
    m_embs = []
    f_ids = []
    m_ids = []
    f_logits = []
    m_logits = []
    
    # For each batch of training data...
    bertflow.eval()
    with torch.no_grad():
        # for step, f_batch in enumerate(female_dataloader):
        #     # print(f_batch.shape)
        #     embs = f_batch[0]
        #     ids = f_batch[1]
        #     logit_v = f_batch[3]
        #     # print(embs.shape)
        #     female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
        #     female_z_list.append(female_z)
        #     f_embs.append(embs)
        #     f_ids.append(ids)
        #     f_logits.append(logit_v)
        for step, m_batch in enumerate(male_dataloader):
            embs = m_batch[0]
            ids = m_batch[1]
            logit_v = m_batch[3]
            # gen = m_batch[2]
            male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
            male_z_list.append(male_z)
            m_embs.append(embs)
            m_ids.append(ids)
            m_logits.append(logit_v)

    new_female_z_list = copy.deepcopy(female_z_list)
    # new_male_z_list = copy.deepcopy(male_z_list)

    # gender_transfer_count = 0
    # semantic_transfer_count = 0
    # non_conversion_count = 0
    # original_old_word = []
    # old_word = []
    # old_id = []
    # new_word = []
    # new_id = []
    # gender = []
    # logit_vals = []

    sum_fem = 0
    count = 0
    for j in range(len(male_z_list)):
        # print("############", j)
        for k in range(len(male_z_list[j][0])):
            sum_fem = sum_fem + male_z_list[j][0][k]
            count+=1
    avg_sum_fem = sum_fem/count
    return avg_sum_fem

In [13]:
import copy
#########generate embeddings in gaussian space#################
def gen_flow_embs(female_dataloader, male_dataloader, avg_sum_fem):
    bertflow.eval()
    
    female_z_list = []
    male_z_list = []
    f_embs = []
    m_embs = []
    f_ids = []
    m_ids = []
    f_logits = []
    m_logits = []
    
    # For each batch of training data...
    bertflow.eval()
    with torch.no_grad():
        for step, f_batch in enumerate(female_dataloader):
            # print(f_batch.shape)
            embs = f_batch[0]
            ids = f_batch[1]
            logit_v = f_batch[3]
            # print(embs.shape)
            female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
            female_z_list.append(female_z)
            f_embs.append(embs)
            f_ids.append(ids)
            f_logits.append(logit_v)
        # for step, m_batch in enumerate(male_dataloader):
        #     embs = m_batch[0]
        #     ids = m_batch[1]
        #     logit_v = m_batch[3]
        #     # gen = m_batch[2]
        #     male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
        #     male_z_list.append(male_z)
        #     m_embs.append(embs)
        #     m_ids.append(ids)
        #     m_logits.append(logit_v)

    new_female_z_list = copy.deepcopy(female_z_list)
    # new_male_z_list = copy.deepcopy(male_z_list)

    gender_transfer_count = 0
    semantic_transfer_count = 0
    non_conversion_count = 0
    original_old_word = []
    old_word = []
    old_id = []
    new_word = []
    new_id = []
    gender = []
    logit_vals = []

    # sum_fem = 0
    # count = 0
    # for j in range(len(male_z_list)):
    #     # print("############", j)
    #     for k in range(len(male_z_list[j][0])):
    #         sum_fem = sum_fem + male_z_list[j][0][k]
    #         count+=1
    # avg_sum_fem = sum_fem/count

    for j in range(len(female_z_list)):
        for k in range(len(female_z_list[j][0])):
        # part1 = torch.tensor([female_z[0][0][15].cpu().detach().numpy() for i in female_z[0][0]])
            # new_male_z_list[j][0][0][k] = female_z_list[j][0][0][k]
            new_female_z_list[j][0][k] = avg_sum_fem
        
            
        eee = bertflow(new_female_z_list[j], reverse=True)
        eee = torch.squeeze(eee)
        ffff = decode_emb(eee)
        ffff
        for key,i in enumerate(decode_emb(f_embs[j][:,:].to(device))):
            # if f_logits[j][key].item() < -10:
            original_old_word.append(vv[f_ids[j][key].item()])
            old_word.append(vv[i.item()])
            old_id.append(i.item())
            new_word.append(vv[ffff[key].item()])
            new_id.append(ffff[key].item())
            gender.append(1)
            logit_vals.append(f_logits[j][key].item())
    
    return original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals

In [14]:
# for i in male_logits:
#     print(i)

In [15]:
from torch.utils.data import TensorDataset



original_old_word = []
old_word = []
old_id = []
new_word = []
new_id = []
gender = []
logit_vals = []


for jj in range(1,2):
    for iteration in range(20000,21000,20000):
        print(iteration)
    
        # with open('../1_viz/attribute_embeddings/embeddings_1000.pkl', 'rb') as f:
        #     embeddings = pickle.load(f)
        # with open('../1_viz/attribute_embeddings/gender_1000.pkl', 'rb') as f:
        #     gen_labels = pickle.load(f)
        # with open('../1_viz/attribute_embeddings/token_ids_1000.pkl', 'rb') as f:
        #     attribute_ids = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/embeddings_'+str(iteration)+'.pkl', 'rb') as f:
            embeddings = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/gender_'+str(iteration)+'.pkl', 'rb') as f:
                gen_labels = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/token_ids_'+str(iteration)+'.pkl', 'rb') as f:
                attribute_ids = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/logits_'+str(iteration)+'.pkl', 'rb') as f:
                logits = pickle.load(f)

    
        start_ind = 0
        end_ind = 0
        # ratio = 10000
        ratio = len(attribute_ids)
        
        for i in range(int(len(attribute_ids)/ratio)):
            print(i)
            end_ind+=ratio
            female_embeddings, female_attribute_ids, female_gen_labels, female_logits, male_embeddings, male_attribute_ids, male_gen_labels, male_logits = get_examples(embeddings[start_ind:end_ind], gen_labels[start_ind:end_ind], attribute_ids[start_ind:end_ind], logits[start_ind:end_ind], 1, train=False)
            female_data_pairs_test = TensorDataset(female_embeddings, female_attribute_ids, female_gen_labels, female_logits)
            male_data_pairs_test = TensorDataset(male_embeddings, male_attribute_ids, male_gen_labels, male_logits)
            
            female_dataloader = DataLoader(
                        female_data_pairs_test,  # The training samples.
                        sampler = None, #RandomSampler(female_data_pairs_test), # Select batches randomly
                        batch_size = 20 # Trains with this batch size.
                    )
            
            male_dataloader = DataLoader(
                        male_data_pairs_test,  # The training samples.
                        sampler = None, #RandomSampler(male_data_pairs_test), # Select batches randomly
                        batch_size = 20 # Trains with this batch size.
                    )
            start_ind += ratio
            # try:
            # print((female_embeddings.shape), (male_embeddings.shape))
            avg_sum_fem = gen_flow_embs1(male_dataloader)
            original_old_word_temp, old_word_temp, old_id_temp, new_word_temp, new_id_temp, gender_temp, logit_vals_temp = gen_flow_embs(female_dataloader, male_dataloader, avg_sum_fem)
            original_old_word+= original_old_word_temp
            old_word+= old_word_temp
            old_id+= old_id_temp
            new_word+= new_word_temp
            new_id+= new_id_temp
            gender+= gender_temp
            logit_vals+= logit_vals_temp
            # except:
            #     original_old_word_temp, old_word_temp, old_id_temp, new_word_temp, new_id_temp, gender_temp, logit_vals_temp
            #     print(len(male_dataloader), len(female_dataloader))
    
        

20000
0


/tmp/ipykernel_78755/1076840909.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits_list = embedding_decoder(torch.tensor(emb))


In [16]:
len(male_dataloader)

777

In [17]:
# embeddings[start_ind:end_ind]

In [18]:
# len(embeddings)

In [19]:
lst = [original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals]

In [20]:
# len(original_old_word)

In [21]:
dict = {'original_old_word': original_old_word, 'old_word': old_word, 'old_id': old_id, 'new_word': new_word, 'new_id': new_id, 'gender': gender, 'logits': logit_vals}
df = pd.DataFrame(dict)
df["count"] = 1 
df

,original_old_word,old_word,old_id,new_word,new_id,gender,logits,count
0,maize,maize,21154,european,2647,1,-10.360230,1
1,corn,corn,9781,corn,9781,1,-9.948413,1
2,african,african,3060,european,2647,1,-21.796511,1
3,ant,ant,14405,ant,14405,1,-10.193929,1
4,bananas,bananas,26191,bananas,26191,1,-10.219945,1
...,...,...,...,...,...,...,...,...
53196,ito,ito,23333,ec,14925,1,-10.753832,1
53197,##we,##we,8545,eu,7327,1,-10.706676,1
53198,gorilla,gorilla,23526,western,2530,1,-16.715841,1
53199,dr,dr,2852,other,2060,1,-10.501709,1


In [22]:
filter = df['original_old_word'].str.contains("##")
df = df[~filter]
filter = df['original_old_word'].str.contains("[CLS]")
df = df[~filter]
filter = df['original_old_word'].str.contains("[SEP]")
df = df[~filter]
filter = df['original_old_word'].str.contains("[PAD]")
df = df[~filter]

In [48]:
# count = 0
# female_final = []
# for index, row in df.iterrows():
#     # try:
#     if (row['original_old_word'] != row['new_word']):
#         female_final.append([row['original_old_word'], row['new_word']])
#         count+=1
#         # print(row['original_old_word'], row['new_word'],count)
#     # except:
#     #     (row['female'], row['male'])
# female_final = pd.DataFrame(female_final)
# female_final = female_final.rename(columns={0: 'original_old_word', 1: 'new_word'})
# female_final

In [47]:
# df_stats_final = female_final.groupby(['original_old_word'])['new_word'].agg(pd.Series.mode).to_frame()
# df_stats_final["original_old_word"] = df_stats_final.index
# df_stats_final

In [46]:
# print(df_stats_final.to_string())

In [26]:
# # df_stats = df
# df_stats_final = pd.DataFrame(female_final.groupby(['original_old_word'])['new_word'].agg(pd.Series.mode))
# # df_stats_final["var"] = pd.DataFrame(female_final.groupby(['original_old_word'])['logits'].agg('var'))
# # df_stats_final["original_word_count"] = female_final.DataFrame(df.groupby(['original_old_word'])['count'].agg('sum'))
# # df_stats_final["original_old_word"] = df_stats_final.index

# df_stats_final = df_stats_final.reset_index(drop=True)
# df_stats_final

,new_word
0,s
1,s
2,eu
3,european
4,european
...,...
851,"[co, d, un]"
852,latin
853,"[english, medieval]"
854,european


In [27]:
# female_final

,original_old_word,new_word
0,maize,european
1,african,european
2,africans,countries
3,africa,europe
4,ethiopia,later
...,...,...
20944,gorilla,western
20945,ito,ec
20946,gorilla,western
20947,dr,other


In [28]:
df.to_csv('female_male_full_pairs_'+attribute_pair+'.csv', index=False)

In [29]:
# df_stats = df
df_stats = pd.DataFrame(df.groupby(['original_old_word'])['logits'].agg('mean'))
df_stats["var"] = pd.DataFrame(df.groupby(['original_old_word'])['logits'].agg('var'))
df_stats["original_word_count"] = pd.DataFrame(df.groupby(['original_old_word'])['count'].agg('sum'))
df_stats["original_old_word"] = df_stats.index

df_stats = df_stats.reset_index(drop=True)

In [30]:
df_stats

,logits,var,original_word_count,original_old_word
0,-9.441541,NaN,1,""""
1,-9.814542,0.522413,22,'
2,-9.547699,0.195233,21,-
3,-9.230884,NaN,1,1966
4,-10.724882,NaN,1,23
...,...,...,...,...
1342,-10.225252,0.653999,9,zu
1343,-14.868702,4.470425,150,zulu
1344,-9.569751,0.189960,11,ʿ
1345,-9.471662,0.172891,38,α


In [31]:
df_stats = df_stats.sort_values('original_word_count', ascending=False)
df_stats.head(50)

,logits,var,original_word_count,original_old_word
452,-12.843046,8.055965,150,east
738,-10.743741,1.366201,150,ki
791,-10.748596,0.799593,150,libya
792,-12.182207,1.830697,150,libyan
782,-11.618973,1.923610,150,leone
783,-10.299981,0.774853,150,leopard
1266,-12.081768,1.683302,150,tunisia
1267,-12.702771,2.227574,150,tunisian
53,-18.893459,5.245274,150,africa
471,-10.008586,0.698231,150,el


In [32]:
df['original_old_word'].value_counts()

original_old_word
east        150
africans    150
saharan     150
somalia     150
ant         150
           ... 
coco          1
boogie        1
lung          1
le            1
out           1
Name: count, Length: 1347, dtype: int64

In [33]:
final_df = pd.DataFrame(df.groupby('original_old_word').new_word.agg(lambda x : x.mode())) # .reset_index()
final_df["original_old_word"] = final_df.index
final_df = final_df.reset_index(drop=True)
final_df

,new_word,original_old_word
0,"""",""""
1,','
2,-,-
3,1966,1966
4,23,23
...,...,...
1342,zu,zu
1343,zulu,zulu
1344,ʿ,ʿ
1345,α,α


In [34]:
df_stats = pd.merge(df_stats, final_df, how='inner', on=['original_old_word'])
df_stats

,logits,var,original_word_count,original_old_word,new_word
0,-12.843046,8.055965,150,east,east
1,-10.743741,1.366201,150,ki,ki
2,-10.748596,0.799593,150,libya,libya
3,-12.182207,1.830697,150,libyan,libyan
4,-11.618973,1.923610,150,leone,leone
...,...,...,...,...,...
1342,-11.858043,NaN,1,band,band
1343,-10.347375,NaN,1,bai,-
1344,-10.035360,NaN,1,bahamas,bahamas
1345,-9.676406,NaN,1,bad,bad


In [35]:
df_stats[df_stats["original_old_word"]=="her"]

,logits,var,original_word_count,original_old_word,new_word


In [36]:
# df_stats = df_stats.dropna()

In [37]:
df_stats = df_stats.sort_values('logits', ascending=True)
df_stats.head(50)

,logits,var,original_word_count,original_old_word,new_word
97,-22.750923,6.333617,150,african,european
101,-19.633844,6.220272,150,afro,european
8,-18.893459,5.245274,150,africa,europe
76,-16.439695,7.698027,150,negro,european
98,-16.015917,5.446305,150,africans,europeans
30,-15.822843,2.192953,150,zimbabwe,european
229,-15.618771,3.360869,63,abyss,western
62,-15.585097,3.786746,150,mali,mali
112,-15.511767,1.655062,147,malawi,malawi
244,-15.244792,2.119563,56,sudanese,"[german, sudanese]"


In [38]:
df_stats.to_csv('female_male_pairs_'+attribute_pair+'.csv', index=False)

In [39]:
df_stats_keep = df_stats[df_stats["original_word_count"]>2]
df_stats_discard = df_stats[df_stats["original_word_count"]<3]

In [40]:
# # df1 = df_stats[['original_old_word','new_word']]
# df_stats[df1.eq(df1.iloc[:, 0], axis=0).all(axis=1)]

In [41]:
df_stats_keep

,logits,var,original_word_count,original_old_word,new_word
97,-22.750923,6.333617,150,african,european
101,-19.633844,6.220272,150,afro,european
8,-18.893459,5.245274,150,africa,europe
76,-16.439695,7.698027,150,negro,european
98,-16.015917,5.446305,150,africans,europeans
...,...,...,...,...,...
822,-9.098877,0.001276,3,bio,bio
837,-9.083066,0.013399,3,islam,"[european, foreign, islam]"
799,-9.083007,0.010672,3,amino,amino
865,-9.067478,0.012462,3,afghan,afghan


In [42]:
print(df_stats_keep.to_string())

        logits        var  original_word_count original_old_word                                       new_word
97  -22.750923   6.333617                  150           african                                       european
101 -19.633844   6.220272                  150              afro                                       european
8   -18.893459   5.245274                  150            africa                                         europe
76  -16.439695   7.698027                  150             negro                                       european
98  -16.015917   5.446305                  150          africans                                      europeans
30  -15.822843   2.192953                  150          zimbabwe                                       european
229 -15.618771   3.360869                   63             abyss                                        western
62  -15.585097   3.786746                  150              mali                                        

In [43]:
print(df_stats_discard.to_string())

         logits       var  original_word_count original_old_word                new_word
1264 -14.729460       NaN                    1             combo                       "
1023 -13.593992       NaN                    1               sai                     sai
1075 -13.301487       NaN                    1         alexandra                       -
934  -12.764111  0.996154                    2          colonies                colonies
918  -12.616779  0.161891                    2               cfa          [cfa, foreign]
976  -12.510983  0.339591                    2              wang              [bu, wang]
1204 -12.436542       NaN                    1                mk                      mk
1266 -12.420897       NaN                    1           chinese                 chinese
1110 -12.267403       NaN                    1          swallows               crusaders
1026 -11.911312       NaN                    1               yer                  allied
1342 -11.858043      

In [44]:
for index, row in df_stats_keep.iterrows():
    try:
        if (row['original_old_word'] != row['new_word']):
            print(row['original_old_word'], row['new_word'])
    except:
        print(row['original_old_word'], row['new_word'])

african european
afro european
africa europe
negro european
africans europeans
zimbabwe european
abyss western
sudanese ['german' 'sudanese']
angola european
ivory ['french' 'ivory']
apartheid colonial
botswana european
nigerian european
johannesburg european
niger ['european' 'niger']
elisabeth ec
mba .
nigeria europe
kala european
rhodesia british
orlando ['orlando' 'the']
mbe sas
senegal french
tunisian european
durban european
sank ["'" 'sank']
mma set
bala european
gambia british
boer british
mule ['##t' ',' 'european' 'mer' 'mule' 'second']
tunisia europe
nwa ['-' 'nwa']
transvaal british
yo european
kenyan british
adi eu
ul european
sierra west
ethiopian european
algerian french
mag european
mg ['fe' 'mg' 'public' 'q']
algeria europe
currie international
indian european
ban european
akin eu
mp european
ama european
molten ['church' 'in' 'molten']
aba european
kane ['ho' 'kane' 'l' 'to' 'two']
afrikaans german
meng ['in' 'meng']
egypt europe
asiatic ['asiatic' 'european']
ek ['ek

In [45]:
df_stats_keep

,logits,var,original_word_count,original_old_word,new_word
97,-22.750923,6.333617,150,african,european
101,-19.633844,6.220272,150,afro,european
8,-18.893459,5.245274,150,africa,europe
76,-16.439695,7.698027,150,negro,european
98,-16.015917,5.446305,150,africans,europeans
...,...,...,...,...,...
822,-9.098877,0.001276,3,bio,bio
837,-9.083066,0.013399,3,islam,"[european, foreign, islam]"
799,-9.083007,0.010672,3,amino,amino
865,-9.067478,0.012462,3,afghan,afghan
